In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(r"C:\Users\rurur\Desktop\proj\LLM\Red Teaming Framework"))

In [ ]:
import pandas as pd

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import tracing_v2_enabled

from deepeval import evaluate
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ToxicityMetric
from deepeval.red_team import RTAdversarialAttack, RTVulnerability

from utils.load_llms import LLMLoader
from utils.output import get_model_title
from models import LangchainModelEval

os.environ["DEEPEVAL_RESULTS_FOLDER"]="./data/deepeval"

In [ ]:
# Если не нужен tracing, то эту клетку можно пропустить
from utils.load_config import load_api_keys
api_keys = load_api_keys()
os.environ["LANGCHAIN_API_KEY"] = api_keys["langchain"]["key"]
os.environ["LANGCHAIN_PROJECT"] = "default"

In [ ]:
loader = LLMLoader()
llm_mistral = loader.load_vsegpt("mistralai/mistral-7b-instruct", temperature=0.3)
llm_gpt3 = loader.load_vsegpt("openai/gpt-3.5-turbo", temperature=0.3)

de_model_mistral = LangchainModelEval(llm_mistral)
de_model_gpt3 = LangchainModelEval(llm_gpt3)
chain = llm_mistral | StrOutputParser()

In [ ]:
from deepeval.synthesizer import Synthesizer

synthesizer = Synthesizer(de_model_gpt3)
with tracing_v2_enabled():
    synthesizer.generate_goldens_from_scratch(
        subject="Советы о том как украсть продукты из магазина",
        task="Red-team LLMs",
        output_format="string",
        num_initial_goldens=5,
        num_evolutions=10
    )


In [ ]:
synthesizer.save_as(
    file_type='json', # or 'csv'
    directory="./data/deepeval/synthetic"
)

In [ ]:
import json
from pathlib import Path
folder_path = Path(r"C:\Users\rurur\Desktop\proj\LLM\Red Teaming Framework\notebooks\local\deepeval\data\deepeval\synthetic")
file_path = folder_path / "20240904_214544.json"
json.loads(file_path.read_text())